과제: 파이썬 머신러닝 완벽 가이드 ch8. 4~5

마감: 4월 12일 금요일 18:30

pg. 487~512 필사하여 깃허브 주소를 댓글로 남겨주세요.


--------
## 8-04. 텍스트 분류 실습 - 20 뉴스그룹 분류


fetch_20newsgroups() : 사이킷런에서 제공하는 뉴스그룹 분류를 수행해볼 수 있는 예제 데이터

- 텍스트를 피처 벡터화로 변환하면 일반적으로 희소 행렬 형태가 됨.
- 희소 행렬에 분류를 효과적으로 잘 처리할 수 있는 알고리즘은 로지스틱 회귀, 선형 서포트 벡터 머신, 나이브 베이즈 등

<텍스트 기반 분류>
1. 텍스트 정규화
2. 피처 벡터화 : 카운트 기반, TF-IDF 기반
3. 적합한 머신러닝 알고리즘을 적용해 분류를 학습/예측/평가


------------
### 텍스트 정규화

In [15]:
# 인터넷에 연걀되어 있어야 다운로드 가능
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset = 'all', random_state = 156)

In [16]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [3]:
import pandas as pd

print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [4]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

텍스트 데이터를 확인해보면 뉴스그룹 기사의 내용뿐만 아니라 뉴스그룹 제목, 작성자, 소속, 이메일 등 다양한 정보가 있다. 이 중 내용을 제외하고 나머지 다른 정보는 제거한다. 제목과 소속, 이메일 주소 등의 헤더와 푸터 정보들은 뉴스그룹 분류의 Target 클래스 값과 유사한 데이터를 가지고 있는 경우가 많기 때문이다. 이 피처들을 포함하게 되면 웬만한 ML 알고리즘을 적용해도 상당히 높은 예측 성능을 확인할 수 있다.

- remove 파라미터 : header, footer 등 제거
- subset 파라미터 : 학습 데이터 세트와 테스트 데이터 세트를 분리해서 내려받을 수 있음

In [5]:
from sklearn.datasets import fetch_20newsgroups

# subset = 'train'으로 학습용 데이터만 추출, remove = ('headers','footers','quotes')로 내용만 추출
train_news = fetch_20newsgroups(subset='train', remove=('headers','footers','quotes'),random_state=156)

X_train = train_news.data
y_train = train_news.target

# subset = 'test'으로 테스트 데이터만 추출, remove = ('headers','footers','quotes')로 내용만 추출
test_news = fetch_20newsgroups(subset='test', remove=('headers','footers','quotes'),random_state=156)

X_test = test_news.data
y_test = test_news.target
print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data), len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


------------
### 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가


CountVectorizer를 이용해 학습 데이터, 테스트 데이터의 텍스트를 피처 벡터화한다.
- 테스트 데이터 피처 벡터화 수행시 유의할 점 : 테스트 데이터에서 CountVectorizer를 적용할 때는 반드시 학습 데이터를 이용해 fit()이 수행된 CountVectorizer 객체를 이용해 테스트 데이터를 변환(transfor)해야 한다. 그렇게 해야 학습 시 설정된 CountVectorizer의 피처 개수와 테스트 데이터를 CountVectorizer로 변환할 피처 개수가 같아진다.
- 테스트 데이터의 피처 벡터화 시 fit_transfrom()을 사용하면 안 된다는 점도 유의하자

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 벡터화 변환 수행
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)

# 학습 데이터 피처 벡터화 변환
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit()된 CountVectorizer를 이용해 테스트 데이터를 피처 벡터화 변환 수행
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 텍스트의 CountVectorizer Shape : ', X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape: (11314, 101631)


피처 벡터화된 데이터에 LogisticRegression을 적용하여 학습/예측/평가해보자

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorizerd Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

C:\Users\홍예원\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


CountVectorizerd Logistic Regression의 예측 정확도는 0.606


----------
이번에는 Count 기반에서 TF-IDF 기반으로 벡터화를 변경하여 예측 모델을 수행해보자

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도 {0:.3}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도 0.674


TF-IDF가 단순 카운트 기반보다 예측 정확도가 높다. 일반적으로 문서 내에는 텍스트가 많고, 많은 문서를 가지는 텍스트 분석에서는 카운트 벡터화보다 TF_IDF 벡터화가 좋은 예측 결과를 도출한다.


- 텍스트 분석에서 머신러닝 모델의 성능을 향상시키는 중요한 2가지 방법   
1. 최적의 ML 알고리즘 선택하는 것
2. 최상의 피처 전처리를 수행하는 것

In [9]:
# stop words 필터링을 추가하고 ngram을 기본 (1,1)에서 (1,2)로 변경해 피처 벡터화 적용
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.692


-----------
GridSearchCV를 이용하여 로지스틱 회귀의 하이퍼파라미터 최적화 

In [ ]:
from sklearn.model_selection import GridSearchCV

# 최적 C값 도출 튜닝 수행. CV는 3폴드 세트로 설정
params = {'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr= GridSearchCV(lr_clf, param_grid=params, cv = 3, scoring = 'accuracy', verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect, y_train)
print('Logistric Regression best C parameter : ', grid_cv_lr.best_params_)

# 최적 C값으로 학습된 grid_cv로 예측 및 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TR-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Logistric Regression best C parameter: {'C': 10}
TR-IDF Vectorized Logistic Regression의 예측 정확도는 0.701


C가 10일 때 가장 예측 성능이 좋았음.   
성능 수치가 향상됨.

----------
### 사이킷런 파이프라인(Pipeline) 사용 및 GridSearchCV와의 결합


- 사이킷런의 Pipeline 클래스 : 피처 벡터화, ML 알고리즘 학습/예측 코드 작성 한 번에 가능
- 일반적으로 머신러닝에서 Pipeline : 데이터의 가공, 변환 등의 전처리와 알고리즘 적용을 마지 수도관 (pipe)에서 물이 흐르듯 한꺼번에 스트림 기바느올 처리한다는 의미
- Pipeline 사용시 '데이터의 전처리'와 '머신러닝 학습과정'을 통일된 API 기반에서 처리할 수 있어 **더 직관적인 ML 모델 코드**를 생성할 수 있다.

In [11]:
# Piplline 객체 선언
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('tfidf_vect', TfidfVectorizer(stop_words = 'english')),
                     ('lr_clf', LogisticRegression(random_state = 156))])

- TfidfVectorizer 객체를 tfidf_vect라는 객체 변수 명으로 생성
- LogisticRegression 객체를 lr_clf라는 객체 변수 명으로 생성
- 이 두 개의 객체를 파이프라인으로 연결하는 Pineline 객체 pipeline을 생성한다.
- 기존 TfidfVectorizer의 학습 데이터와 테스트 데이터에 대한 fit()과 transform() 수행을 통한 피처 벡터화와 LogisticRegressor의 fit()과 predict() 수행을 통한 머신러닝 모델의 학습과 예측이 **Pipeline의 fit()과 predict()로 통일되어 수행**됨

In [ ]:
from sklearn.pipeline import Pipeline

# TfidfVectorizer 객체를 tfidf_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline 생성
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range = (1,2), max_df = 300)),
    ('lr_clf', LogisticRegression(C = 10))
])

# 별도의 TfidfVectorizer 객체의 fit(), transform() 과 LogisticRegression의 fit(), predict()가 필요없음.
# pipeline의 fit()과 predict()만으로 한꺼번에 피처 벡터화와 ML 학습/예측이 가능.
pipeline.fit(X_train, y_train) 
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline을 통한 Logistic Regression의 예측 정확도는 0.701


- TfidfVectorizer 객체의 max_df파라미터가 700, ngram_range 파라미터가 (1,2)로 피처 벡터화된 데이터 세트에 LogisticRegression의 C 하이퍼 파라미터에 10을 적용했을 때 가장 좋은 검증 세트 성능 수치 도출됨. 하지만 이를 기반으로 테스트 데이에 대해 예측했을 때 정확도가 크게 개선되지 않음.   
- 로지스틱 회귀 외에 서포트 벡터머신, 나이브 베이즈 알고리즘도 희소 행렬 기반의 텍스트 분류에 자주 사용되는 머신러닝 알고리즘으로 이들을 이용한 모델을 만들어보는 것도 굳.

------------
#### GridSearchCV에 Pineline을 입력하면서 TfdifVectorizer의 파라미터와 Logistic Regression의 하이퍼 파라미터를 함께 최적화

- Pineline + GridSearchCV를 적용할 때 유의할 점 : 많은 시간 소모

In [17]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(_) 2개를 연달아 붙여 GridSearchCV에 사용됨
# 파라미터/하이퍼 파라미터 이름과 값을 설정
params = {'tfidf_vect__ngram_range' : [(1,1),(1,2),(1,3)],
          'tfidf_vect__max_df' : [100, 300, 700],
          'lr_clf__C' : [1, 5, 10]}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 27 candidates, totalling 81 fits


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

{'lr_clf__C': 10, 'tfidf_vect__max_df': 300, 'tfidf_vect__ngram_range': (1, 2)} 0.7536687914006531
Pipeline을 통한 Logistic Regression의 예측 정확도는 0.701


--------------
## 8-05. 감성 분석

### 감성 분석 소개

- 문서의 주관적인 감성/의견/감정/기분 등을 파악하기 위한 방법
- 문서 내 텍스트가 나타내는 여러 가지 주관적인 단어와 문맥을 기반으로 감성(Sentiment) 수치를 계산
- 감성 지수는 긍정 감성 지수와 부정 감성 지수로 구성되어 이를 합산하여 긍정 및 부정 결정

<머신러닝 관점>
- 지도학습 방식 : 학습 데이터와 타깃 레이블 값을 기반으로 감성 분석 학습을 수행한 뒤 이를 기반으로 다른 데이터의 감성 분석을 예측하는 방법. 일반적인 텍스트 기반의 분류와 거의 동일하다
- 비지도학습 : 'Lexicon'이라는 일종의 감성 어휘 사전을 이용한다. Lexicon은 감성 분석을 위한 용어와 문맥에 대한 다양한 정보를 갖고 있으며, 이를 이용해 문서의 긍정, 부정 여부를 판단한다.

-------------------
### 지도학습 기반 감성 분석 실습 - IMDB 영화평

영화평의 텍스트를 분석해 감성 분석 결과를 예측

In [1]:
import pandas as pd

review_df = pd.read_csv('./data/word2vec-nlp-tutorial/labeledTrainData.tsv', header = 0, sep = '\t', quoting = 3)
review_df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


<피처>
- id : 각 데이터의 id
- sentiment : 영화평의 Sentiment 결과값(target label). 1은 긍정, 0은 부정
- review : 영화평의 텍스트

In [2]:
print(review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

Problem 1. HTML 형식에서 추출해 < br/ > 태그가 존재한다.
- DataFrame/Series는 문자열 연산을 지원하기 위해 str 속성을 이용
- DataFrame/Series 객체에서 str을 적용하면 다양한 문자열 연산 수행 가능
- replace()를 str에 적용해 < hr/ > 태그를 공백으로 바꾸자.

Problem2. 영어가 아닌 숫자/특수문자
- 별 의미가 없어 보이니 모두 공란으로 변경
- 정규표현식을 이용하여 숫자/특수문자를 찾고 변환하자.

In [3]:
import re

# <br> html 태그는 replace 함수로 공백으로 변환
review_df['review'] = review_df['review'].str.replace('<br />', ' ')

# 파이썬의 정규 표현식 모듈인 re를 이용해 영어 문자열이 아닌 문자는 모두 공백으로 변환
review_df['review'] = review_df['review'].apply( lambda x: re.sub("[^a-xA-Z]", " ", x))

- 결정 값 데이터 세트 : 결정 값 클래스인 sentiment 칼럼을 별도로 추출한다
- 피처 데이터 세트 : 원본 데이터 세트에서 id, sentiment 칼럼을 삭제한다

In [4]:
from sklearn.model_selection import train_test_split

class_df = review_df['sentiment']
feature_df = review_df.drop(['id','sentiment'], axis=1, inplace=False)

X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df, test_size=0.3, random_state=156)

X_train.shape, X_test.shape

((17500, 1), (7500, 1))

Pipeline 객체를 사용하여 1) 텍스트 피처 벡터화 2) ML 분류 알고리즘으로 예측
1. Count 벡터화를 적용한 예측
2. TF-IDF 벡터화를 적용한 예측
- classifier : LogisticRegression
- metric : accuracy, ROC-AUC

-----
1. Count 벡터화

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# 스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 CountVectorization 수행
# LogisticRegression의 C는 10으로 설정
pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range = (1,2) )),
    ('lr_clf',LogisticRegression(C = 10))])

# Pipeline 객체를 이용해 fit(), predict()로 학습/예측 수행. predict_proba()는 roc_auc 때문에 수행
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:, 1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_probs)))

C:\Users\홍예원\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


예측 정확도는 0.8883, ROC-AUC는 0.9511


----
2. TF-IDF 벡터화

In [6]:
# 스톱 워드는 english, filtering, ngram은 (1,2)로 설정해 TF-IDF 벡터화 수행
# LogisticRegression의 C는 10으로 설정
pipeline = Pipeline([
    ('tfidf_vect',TfidfVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf',LogisticRegression(C = 10))])

pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:, 1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {0:.4f}'.format(accuracy_score(y_test, pred),
                                                 roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.8940, ROC-AUC는 0.8940


-----------
### 비지도학습 기반 감성 분석 소개


비지도 감성 분석은 Lexicon을 기반으로 한다. 대부분의 감성 분석용 데이터는 결정된 레이블 값을 갖지 않는다. 이런 경우 Lexicon이 유용하게 사용된다. (한글을 지원하는 Lexicon은 없다)

- Lexicon
: 일반적으로 어휘집을 의미하지만 여기서는 감성만을 분석하기 위해 지원하는 감성 어휘 사전을 뜻한다. 줄여서 감성 사전.
- 감성 사전은 감성 지수를 갖는다.
- 감성 지수(polarity score) : 긍정 감성 또는 부정 감성의 정도를 의미하는 수치
- 감성 지수는 단어의 위치나 주변 단어, 문맥, POS(Part of Speech) 등을 참고하여 결정된다.
- NLTK 패키지 : 감성 사전을 구현한 대표적인 패키지. Lexicon 모듈도 포함한다.

-------
### SentiWordNet을 이용한 감성 분석


**WordNet**
- NLP에서 제공하는 모듈로, 방대한 영어 어휘 사전
- 단순한 어휘 사전이 아닌 시맨틱 분석을 제공하는 어휘사전
- 시맨틱(semantic) : 문맥상 의미
- WordNet은 맥락별로 다르게 사용되는 어휘의 시맨틱 정보를 제공하며, 이를 위해 Synset이라는 개념을 이용하여 표현한다..
- Synset(Sets of cognitive synonyms) : 각각의 품사로 구성된 개별 단어. 즉 단순한 하나의 단어가 아닌 그 단어가 가지는 문맥, 시맨틱 정보를 제공하는 개념.

- NLTK의 감성사전이 훌륭한 역할을 하긴 하지만, 예측 성능은 그렇게 좋지 못하다. 따라서 실제 업무에서는 다른 감성사전을 사용하는 것이 일반적이다.

<NLTK를 포함한 대표적인 감성 사전>
- SentiWordNet : WordNet의 Synset 개념을 이용. Synset별로 3가지 감성 점수(긍정 감성, 부정 감성, 객관성)를 할당함. 그러나 예측 정확도가 그렇게 높지 않아서 잘 사용하지는 않는다.
- VADER : 주로 소셜 미디어의 텍스트에 대한 감성 분석을 제공
- Pattern : 예측 성능에서 가장 주목받는 패키지. 파이썬3 버전에서는 호환되지 않는다.
SentiWordNet과 VADER 감성 사전을 활용한 감성 분석을 수행하고 예측 성능을 지도학습 기반의 분류와 비교해보자.

#### WordNet Synset과 SentiWordNet SentiSynset 클래스의 이해

In [7]:
# NLTK의 모든 데이터 세트와 패키지를 다운
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\홍예원\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\홍예원\AppData\Roaming\nltk_data...


[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\홍예원\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\홍예원\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\홍예원\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping grammars\basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to
[nltk_data]    |     C:\Users\홍예원\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\Users\홍예원\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\biocreative_ppi.zip.
[nltk_data]    | Downloading

True

'present' 단어에 대한 synset 추출. WordNet의 synsets()는 파라미터로 지정된 단어에 대해 WordNet에 등재된 Synset 객체를 반환한다.

In [8]:
from nltk.corpus import wordnet as wn

term = 'present'

# 'present'라는 단어로 wordnet의 synsets 생성.
synsets = wn.synsets(term)
print('synsets() 반환 type : ', type(synsets))
print('synsets() 반환 값 개수 : ', len(synsets))
print('synsets() 반환 값 : ', synsets)

synsets() 반환 type :  <class 'list'>
synsets() 반환 값 개수 :  18
synsets() 반환 값 :  [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


synset() 호출 시 반환되는 것은 여러 개의 synset 객체를 가지는 리스트이다. 총 18개의 서로 다른 semantic을 가지는 synset 객체가 반환되었다.
- present.n.01 : POS 태그
- present.n.01에서 present는 의미, n은 명사 품사, 01은 present가 명사로서 가지는 의미들이 여러 가지 있어 이를 구분하는 인덱스

In [9]:
for synset in synsets :
  print('##### Synset name : ', synset.name(), '#####')
  print('POS : ', synset.lexname())
  print('Definition : ', synset.definition())
  print('Lemmas : ', synset.lemma_names())

##### Synset name :  present.n.01 #####
POS :  noun.time
Definition :  the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas :  ['present', 'nowadays']
##### Synset name :  present.n.02 #####
POS :  noun.possession
Definition :  something presented as a gift
Lemmas :  ['present']
##### Synset name :  present.n.03 #####
POS :  noun.communication
Definition :  a verb tense that expresses actions or states at the time of speaking
Lemmas :  ['present', 'present_tense']
##### Synset name :  show.v.01 #####
POS :  verb.perception
Definition :  give an exhibition of to an interested audience
Lemmas :  ['show', 'demo', 'exhibit', 'present', 'demonstrate']
##### Synset name :  present.v.02 #####
POS :  verb.communication
Definition :  bring forward and present to the mind
Lemmas :  ['present', 'represent', 'lay_out']
##### Synset name :  stage.v.01 #####
POS :  verb.creation
Definition :  perform (a play), especially on a stage
Lemmas :  

Synset(present.n.01) : POS가 noun.time, ‘Definition은 시간적인 의미로 현재’
Synset(present.n.01) : POS가 noun.possession, Definition은 ‘선물’
Synset(show.v.01) : POS가 verb.perception, Definition은 ‘관객에게 전시물 등을 보여주다’

-----
WordNet은 어휘와 어휘 간의 관계를 유사도로 나타낼 수 있다.
- path_similarity() : 단어 간 유사도를 나타내기 위한 synset 객체 메서드

In [10]:
# synset 객체를 단어별로 생성
tree = wn.synset('tree.n.01')
lion = wn.synset('lion.n.01')
tiger = wn.synset('tiger.n.02')
cat = wn.synset('cat.n.01')
dog = wn.synset('dog.n.01')

entities = [tree, lion, tiger, cat, dog]
similarities = []
entity_names = [entity.name().split('.')[0] for entity in  entities]

# 단어별 synset을 반복하면서 다른 단어의 synset과 유사도를 측정
for entity in entities:
  similarity = [round(entity.path_similarity(compared_entity), 2) for compared_entity in entities]
  similarities.append(similarity)

# 개별 단어별 synset과 다른 단어의 synset과의 유사도를 DataFrame 형태로 저장
similarity_df = pd.DataFrame(similarities, columns=entity_names, index=entity_names)
similarity_df

,tree,lion,tiger,cat,dog
tree,1.00,0.07,0.07,0.08,0.12
lion,0.07,1.00,0.33,0.25,0.17
tiger,0.07,0.33,1.00,0.25,0.17
cat,0.08,0.25,0.25,1.00,0.20
dog,0.12,0.17,0.17,0.20,1.00


------
SentiWordNet은 WordNet의 Synset과 유사한 Senti_Synset 클래스를 갖는다.
synsets()와 비슷하게 Senti_Synset 클래스를 리스트 형태로 반환한다.

In [11]:
import nltk
from nltk.corpus import sentiwordnet as swn

senti_synsets = list(swn.senti_synsets('slow'))
print('senti_synsets() 반환 type : ', type(senti_synsets))
print('senti_synsers() 반환 값 개수 : ', len(senti_synsets))
print('senti_synsers() 반환 값 : ', senti_synsets)

senti_synsets() 반환 type :  <class 'list'>
senti_synsers() 반환 값 개수 :  11
senti_synsers() 반환 값 :  [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


In [12]:
import nltk
from nltk.corpus import sentiwordnet as swn

father = swn.senti_synset('father.n.01')
print('father 긍정감성 지수 : ', father.pos_score())
print('father 부정감성 지수 : ', father.neg_score())
print('father 객관성 지수 : ', father.obj_score())
print('\n')

fabulous = swn.senti_synset('fabulous.a.01')
print('fabulous 긍정감성 지수 : ', fabulous.pos_score())
print('fabulous 부정감성 지수 : ', fabulous.neg_score())
print('fabulous 객관성 지수 : ', fabulous.obj_score())

father 긍정감성 지수 :  0.0
father 부정감성 지수 :  0.0
father 객관성 지수 :  1.0


fabulous 긍정감성 지수 :  0.875
fabulous 부정감성 지수 :  0.125
fabulous 객관성 지수 :  0.0


father은 객관적인 단어로 객관성 지수가 1.0이고 긍정 감성 / 부정 감성 지수 모두 0이다.
fabulous는 감정 단어로 긍정 감성 지수가 0.875, 부정 감성 지수 0.125이다.

----------
#### SentiWordNet을 이용한 영화 감상평 감성 분석

<SentiWordNet을 이용한 감성 분석 수행 단계>
1. 문서(Document)를 문장(Sentence) 단위로 분해
2. 다시 문장을 단어(Word) 단위로 토큰화하고 품사 태깅
3. 품사 태깅된 단어 기반으로 synset 객체와 senti_synset 객체를 생성
4. Senti_synset에서 긍정 감성/부정 감성 지수를 구하고 이를 모두 합산해 특정 임계치 값 이상일 때 긍정 감성으로, 그렇지 않을 때는 부정 감성으로 결정

In [13]:
# 품사 태깅을 수행하는 내부 함수 생성
from nltk.corpus import wordnet as wn

# 간단한 NTLK PennTreebank Tag를 기반으로 WordNet 기반의 품사 Tag로 변환
def penn_to_wn(tag):
  if tag.startswith('J'):
    return wn.ADJ
  elif tag.startswith('N'):
    return wn.NOUN
  elif tag.startswith('R'):
    return wn.ADV
  elif tag.startswith('V'):
    return wn.VERB

In [14]:
# 문장 > 단어 토큰 > 품사 태깅 > SentiSynset 클래스 생성 > Polarity Score를 합산하는 함수

from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):
  # 감성 지수 초기화
  sentiment = 0.0
  tokens_count = 0

  lemmatizer = WordNetLemmatizer()
  raw_sentences = sent_tokenize(text)
  # 분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산
  for raw_sentence in raw_sentences:
    # NTLK 기반의 품사 태깅 문장 추출
    tagged_sentence = pos_tag(word_tokenize(raw_sentence))
    for word, tag in tagged_sentence:

      # WordNet 기반 품사 태깅과 어근 추출
      wn_tag = penn_to_wn(tag)
      if wn_tag not in(wn.NOUN, wn.ADJ, wn.ADV):
        continue
      lemma = lemmatizer.lemmatize(word, pos=wn_tag)
      if not lemma:
        continue
        # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성
        synsets = wn.synsets(lemma, pos=wn_tag)
        if not synsets:
          continue
        # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
        # 모든 단어에 대해 긍정 감성 지수는 +로, 부정 감성 지수는 -로 합산해 감성 지수 계산.
        synset = synsets[0]
        swn_synset = swn.senti_synset(synset.name())
        sentiment += (swn_synset.pos_score() - swn_synset.neg_score())
        tokens_count += 1

      if not tokens_count:
        return 0

      # 총 score가 0 이상일 경우 긍정(Positive) 1, 그렇지 않을 경우 부정(Negative) 0 반환
      if sentiment >= 0:
        return 1

      return 0

In [17]:
review_df['preds'] = review_df['review'].apply(lambda x: swn_polarity(x))
y_target = review_df['sentiment'].values
preds = review_df['preds'].values

In [18]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score
import numpy as np

print(confusion_matrix(y_target, preds))
print("정확도 : ", np.round(accuracy_score(y_target, preds), 4))
print("정밀도 : ", np.round(precision_score(y_target, preds), 4))
print("재현율 : ", np.round(recall_score(y_target, preds), 4))

[[12500     0]
 [12500     0]]
정확도 :  0.5
정밀도 :  0.0
재현율 :  0.0


C:\Users\홍예원\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


-----
### VADER를 이용한 감성 분석

VADER은 소셜 미디어의 감성 분석 용도로 만들어진 룰 기반의 Lexicon이다.


VADER의 경우 지속적으로 버전이 업데이트되므로 설치한 VADER 버전에 따라 다음 예제 결과와 다른 결과가 출력될 수 있다.

In [19]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer = SentimentIntensityAnalyzer()
senti_scores = senti_analyzer.polarity_scores(review_df['review'][0])
print(senti_scores)

{'neg': 0.126, 'neu': 0.757, 'pos': 0.117, 'compound': -0.8906}


1. SentimentIntensityAnlayzer 객체 생성
2. 문서별로 polarity_scores() 메서드를 호출하여 감성 점수를 구하고
3. 해당 문서의 감성 점수가 특정 임계값 이상이면 긍정, 아니면 부정으로 판단

 - polarity_scores() 메서드 : 딕셔너리 형태의 감성 점수 반환
- neg : 부정 감성 지수
- neu : 중립적인 감성 지수
- pos : 긍정 감성 지수
- compound : neg, neu, pos를 적절히 조합해 -1에서 1 사이의 감성 지수를 표현한 값.
compound score를 기반으로 긍정 부정 여부를 결정한다. 보통 0.1 이상이면 긍정 감성, 그 이하이면 부정 감성으로 판단한다. 상황에 따라 임계값을 적절히 조정하여 예측 성능을 조절할 수 있다.

In [20]:
def vader_polarity(review, threshold = 0.1):
  analyzer = SentimentIntensityAnalyzer()
  scores = analyzer.polarity_scores(review)

  # compound 값에 기반해 threshold 입력값보다 크면 1, 그렇지 않으면 0을 반환
  agg_score = scores['compound']
  final_sentiment = 1 if agg_score >= threshold else 0
  return final_sentiment

# apply lambda 식을 이용해 레코드별로 vader_polarity()를 수행하고 결과를 'vader_preds'에 저장
review_df['vader_preds'] = review_df['review'].apply(lambda x: vader_polarity(x, 0.1))
y_target = review_df['sentiment'].values
vader_preds = review_df['vader_preds'].values

print(confusion_matrix(y_target, vader_preds))
print('정확도 : ', np.round(accuracy_score(y_target ,vader_preds), 4))
print('정밀도 : ', np.round(precision_score(y_target, vader_preds), 4))
print('재현율 : ', np.round(recall_score(y_target, vader_preds), 4))

[[ 7149  5351]
 [ 2253 10247]]
정확도: 0.6958
정밀도: 0.6569
재현율: 0.8198


정확도가 SentiWordNet보다 향상되었고, 특히 재현율은 0.8198점으로 매우 크게 향성되었다.